# RFM ile Müşteri Segmentasyonu (Customer Segmentation with RFM)

1. İş Problemi (Business Problem)
2. Veriyi Anlama (Data Understanding)
3. Veri Hazırlama (Data Preparation)
4. RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)
5. RFM Skorlarının Hesaplanması (Calculating RFM Scores)
6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)
7. Tüm Sürecin Fonksiyonlaştırılması

In [1]:
# 1. İş Problemi (Business Problem)
# Bir e-ticaret şirketi müşterilerini segmentlere ayırıp bu segmentlere göre
# pazarlama stratejileri belirlemek istiyor.

In [2]:
# Online Retail II isimli veri seti İngiltere merkezli online bir satış mağazasının
# 01/12/2009 - 09/12/2011 tarihleri arasındaki satışlarını içeriyor.

In [3]:
import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
df_=pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2009-2010")
df=df_.copy()

In [5]:
#asagıdaki gibi aynı faturada bircok ürün var price bunların birim fiyatı
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [6]:
df.shape

(525461, 8)

In [7]:
#müşteri ıd yok ise bu müşteri özelinde bir öneri sunamayız o yüzden eksik değerleri silmeyi tercih edicez.
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [8]:
df["Description"].nunique()

4681

In [9]:
#her bir üründen kaçar tane satılmış bakalım
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3549
REGENCY CAKESTAND 3 TIER              2212
STRAWBERRY CERAMIC TRINKET BOX        1843
PACK OF 72 RETRO SPOT CAKE CASES      1466
ASSORTED COLOUR BIRD ORNAMENT         1457
Name: Description, dtype: int64

In [10]:
#yukarıdaki ürünlerden toplam kaçar adet satış olmus bakalım
#quantity de sorun cıktı
df.groupby("Description").agg({"Quantity":"sum"}).head()

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,179
3 STRIPEY MICE FELTCRAFT,690


In [11]:
#yukarıdaki sekilde görüntüleyemediğimiz için bu forma getirdik
#burada essiz ürün sayısılarından toplamda kaçar adet satılmıs
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,57733
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54698
BROCADE RING PURSE,47647
PACK OF 72 RETRO SPOT CAKE CASES,46106
ASSORTED COLOUR BIRD ORNAMENT,44925


In [12]:
#toplamda kaç tane essiz fatura kesilmiş
df["Invoice"].nunique()


28816

In [13]:
#aynı faturada birden fazla ürün oldugu için o faturanın toplam ücretini bilmiyoruz. 
#fiyat ile quantity çarparsak o ürün için ödenen fiyatı gözlemleriz
df["TotalPrice"]=df["Quantity"]*df["Price"]
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [14]:
#fatura basına toplam ödenen tutarı bulmak istersek:
df.groupby("Invoice").agg({"TotalPrice":"sum"}).head()

,TotalPrice
Invoice,
489434,505.300
489435,145.800
489436,630.330
489437,310.750
489438,2286.240


Veri Hazırlama (Data Preparation)

In [15]:
#veri seti525.461 oldukca zengin oldugu için eksik verileri silebiliriz.
#inplace=True ise kalıcı değişiklik için
df.dropna(inplace=True)

In [16]:
#C ile baslayan iade faturalar oldugu için - bazı quantitty değerleri dönmüştü
#asagıdaki toplada min değerleri max değerleri aykırı
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,417534.000,12.759,101.220,-9360.000,2.000,4.000,12.000,19152.000
Price,417534.000,3.888,71.132,0.000,1.250,1.950,3.750,25111.090
Customer ID,417534.000,15360.645,1680.811,12346.000,13983.000,15311.000,16799.000,18287.000
TotalPrice,417534.000,19.994,99.916,-25111.090,4.250,11.250,19.350,15818.400


In [17]:
#bu sebebple iade olan faturaları çıkaralım tekrar gözlemleyelim.
#önce C olanları gözlemleyelim total price alanı - oldugundan veri setinin yapısında bozulmaya sebep oluyor.
df[df["Invoice"].str.contains("C", na=False)]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.950,16321.000,Australia,-35.400
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.650,16321.000,Australia,-9.900
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.250,16321.000,Australia,-17.000
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.100,16321.000,Australia,-12.600
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.950,16321.000,Australia,-35.400
...,...,...,...,...,...,...,...,...,...
524695,C538123,22956,36 FOIL HEART CAKE CASES,-2,2010-12-09 15:41:00,2.100,12605.000,Germany,-4.200
524696,C538124,M,Manual,-4,2010-12-09 15:43:00,0.500,15329.000,United Kingdom,-2.000
524697,C538124,22699,ROSES REGENCY TEACUP AND SAUCER,-1,2010-12-09 15:43:00,2.950,15329.000,United Kingdom,-2.950
524698,C538124,22423,REGENCY CAKESTAND 3 TIER,-1,2010-12-09 15:43:00,12.750,15329.000,United Kingdom,-12.750


In [18]:
#df e bu C leri çıkarılmıs bir sekilde atayalım
df=df[~df["Invoice"].str.contains("C", na=False)]

RFM Metriklerinin Hesaplanması (Calculating RFM Metrics)

In [19]:
df["InvoiceDate"].max()

Timestamp('2010-12-09 20:01:00')

In [20]:
#rastgele bir tarih giricez today_date atama yapıcaz
#ve tip olarak bunu tarih tipinde tutmasını isticez.
#zaman acısından fark alabilmemize yardımcı olacak
today_date = dt.datetime(2010, 12, 11)
type(today_date)

datetime.datetime

In [21]:
#aslında rfm analizinin temeli basit bir pandas operasyonudur.,
#tüm müşterilere göre groupby a alıcaz-rfm hesaplıcaz
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
rfm.head()

,InvoiceDate,Invoice,TotalPrice
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [22]:
#rfm değişken kolon isimlerini belirtelim cünkü bu orijinal datasetten farklı
rfm.columns = ['recency', 'frequency', 'monetary']

In [23]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


In [24]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4314.000,91.270,96.944,1.000,18.000,53.000,136.000,374.000
frequency,4314.000,4.454,8.169,1.000,1.000,2.000,5.000,205.000
monetary,4314.000,2047.289,8912.523,0.000,307.950,705.550,1722.802,349164.350


In [25]:
#bunlardan skor olusturacagımız için:
#daha öncekiler biraz muhasebe gibiydi
#bizim amacımız müşteri segmentasyonu oldugu için tekilleştirme yaptık müşteri ıd göre
#o yüzden artık yeni veri setimiz rfm 
rfm = rfm[rfm["monetary"] > 0]

In [26]:
#böylece rfm metriklerini elde ettik ve skorlara dönüştürmek için hazırladık
rfm.shape

(4312, 3)

In [27]:
rfm.head()

,recency,frequency,monetary
Customer ID,,,
12346.000,165,11,372.860
12347.000,3,2,1323.320
12348.000,74,1,222.160
12349.000,43,3,2671.140
12351.000,11,1,300.930


RFM Skorlarının Hesaplanması (Calculating RFM Scores)

Tüm metrikler için ayrı ayrı skor hesaplayalım

In [28]:
#recency bizim için küçük olanlar iyiydi sıcak müşterileri temsil ettiği için.f ve m de büyüklük iyiydi
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm.head()
#recency metriği bizim için düşük olmalı skor tablosunda ise 5 max aldık.
#bu sebebple de recency metriği büyük olanın skoru düşük oldu. f ve m den farklı oldugu için açıklama eklendi.

,recency,frequency,monetary,recency_score
Customer ID,,,,
12346.000,165,11,372.860,2
12347.000,3,2,1323.320,5
12348.000,74,1,222.160,2
12349.000,43,3,2671.140,3
12351.000,11,1,300.930,5


In [29]:
#monetary score ile metriği arasında doğru bir orantı oldugu için bunun label etiketleri 1 den 5 e gitti
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score
Customer ID,,,,,
12346.000,165,11,372.860,2,2
12347.000,3,2,1323.320,5,4
12348.000,74,1,222.160,2,1
12349.000,43,3,2671.140,3,5
12351.000,11,1,300.930,5,2


In [30]:
#frekans sıklığı sebebiyle 5 e bölemediğimiz için rank metoduna ihtiyacımız var
rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score
Customer ID,,,,,,
12346.000,165,11,372.860,2,2,5
12347.000,3,2,1323.320,5,4,2
12348.000,74,1,222.160,2,1,1
12349.000,43,3,2671.140,3,5,3
12351.000,11,1,300.930,5,2,1


In [31]:
#rfm skoru için recency ve frequency e bakıyorduk bizim için bu ikisi öncelikliydi
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))

In [32]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12346.000,165,11,372.860,2,2,5,25
12347.000,3,2,1323.320,5,4,2,52
12348.000,74,1,222.160,2,1,1,21
12349.000,43,3,2671.140,3,5,3,33
12351.000,11,1,300.930,5,2,1,51


In [33]:
#string ifade oldugundan asagıdaki listede scorelar yer almadı
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4312.000,91.173,96.861,1.000,18.000,53.000,136.000,374.000
frequency,4312.000,4.456,8.170,1.000,1.000,2.000,5.000,205.000
monetary,4312.000,2048.238,8914.481,2.950,307.988,706.020,1723.142,349164.350


In [34]:
#mesela camphion olan müşterimiz için
rfm[rfm["RFM_SCORE"]=="55"]

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE
Customer ID,,,,,,,
12415.000,11,7,19543.840,5,5,5,55
12431.000,9,13,4370.520,5,5,5,55
12471.000,10,49,20139.740,5,5,5,55
12472.000,5,13,11308.480,5,5,5,55
12474.000,14,13,5048.660,5,5,5,55
...,...,...,...,...,...,...,...
18225.000,1,15,7545.140,5,5,5,55
18226.000,14,15,6650.830,5,5,5,55
18229.000,2,10,3526.810,5,5,5,55


In [35]:
#rfm için isimlendirmeler yaparsak müşteri kitlelerine daha rahat gözlemleyebiliriz

6. RFM Segmentlerinin Oluşturulması ve Analiz Edilmesi (Creating & Analysing RFM Segments)

In [ ]:
#r ve f nin kesişimlerinde sektörde kabul gören bazı sınıflar mevcut(sadık müşteri, riskli, kaybetmemiz gereken vb)

In [37]:
# regex r'[rfm score 1.ögesi][rfm scoreun 2.ögesi] yada direkt 33 gibi 

# RFM isimlendirmesi
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)

In [38]:
rfm.head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,2,5,25,cant_loose
12347.000,3,2,1323.320,5,4,2,52,potential_loyalists
12348.000,74,1,222.160,2,1,1,21,hibernating
12349.000,43,3,2671.140,3,5,3,33,need_attention
12351.000,11,1,300.930,5,2,1,51,new_customers


In [39]:
#bu segmentler hakkında bazı depertmanlar bildirilmeli
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])
#at_risk içinde 611 kişi var ve ortalama 152 gündür yoklar

recency       frequency       monetary      
                       mean count      mean count     mean count
segment                                                         
about_to_sleep       53.819   343     1.201   343  441.320   343
at_Risk             152.159   611     3.074   611 1188.878   611
cant_loose          124.117    77     9.117    77 4099.450    77
champions             7.119   663    12.554   663 6852.264   663
hibernating         213.886  1015     1.126  1015  403.978  1015
loyal_customers      36.287   742     6.830   742 2746.067   742
need_attention       53.266   207     2.449   207 1060.357   207
new_customers         8.580    50     1.000    50  386.199    50
potential_loyalists  18.793   517     2.017   517  729.511   517
promising            25.747    87     1.000    87  367.087    87

In [43]:
#örneğin yukarıdaki tabloda yeni müşteriler ile şampiyonların rec değerleri yakın,bunları birbirinden ayıran ise freq değeri
#need attention grubu ilgi görmezse churn e gider önce uyku sonra terk gibi :)
#depertman hangi sınıfa istiyorsa ilgili depertmana gönderir veya veritabanına yazarız onlar da ilgil kişilere erişir.
#erişim örneği olarak kaybetmememiz gereken müşteriler için 
rfm[rfm["segment"]=="cant_loose"].head()

,recency,frequency,monetary,recency_score,monetary_score,frequency_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.000,165,11,372.860,2,2,5,25,cant_loose
12380.000,101,7,6951.490,2,5,5,25,cant_loose
12482.000,212,29,23691.400,1,5,5,15,cant_loose
12510.000,95,7,4195.450,2,5,5,25,cant_loose
12891.000,94,8,509.500,2,3,5,25,cant_loose


In [44]:
#cant_loose a ait index bilgilerini istersek
rfm[rfm["segment"] == "cant_loose"].index

Float64Index([12346.0, 12380.0, 12482.0, 12510.0, 12891.0, 12932.0, 13044.0,
              13313.0, 13680.0, 13782.0, 13799.0, 13856.0, 14025.0, 14063.0,
              14160.0, 14221.0, 14548.0, 14607.0, 14685.0, 14745.0, 15003.0,
              15013.0, 15015.0, 15125.0, 15141.0, 15222.0, 15306.0, 15321.0,
              15359.0, 15369.0, 15372.0, 15443.0, 15538.0, 15607.0, 15633.0,
              15722.0, 15751.0, 15754.0, 15768.0, 15911.0, 15912.0, 16027.0,
              16032.0, 16158.0, 16177.0, 16197.0, 16335.0, 16467.0, 16631.0,
              16742.0, 16743.0, 16875.0, 16986.0, 17021.0, 17032.0, 17092.0,
              17113.0, 17157.0, 17188.0, 17230.0, 17268.0, 17426.0, 17448.0,
              17454.0, 17512.0, 17578.0, 17602.0, 17651.0, 17940.0, 17969.0,
              17988.0, 18009.0, 18051.0, 18064.0, 18094.0, 18251.0, 18258.0],
             dtype='float64', name='Customer ID')

In [47]:
#şimdi bunları dısarı aktaralım
new_df=pd.DataFrame()
new_df["new_customer_id"] = rfm[rfm["segment"] == "new_customers"].index
#görüntü çirkinliğini düzeltmek adına
new_df["new_customer_id"] = new_df["new_customer_id"].astype(int)
new_df.head()

,new_customer_id
0,12351
1,12385
2,12386
3,12427
4,12441


In [50]:
#bunu csv olarak gönderelim
new_df.to_csv("new_customers.csv")
#bu verilere uygun yeni müşteri yapabilirler yada bunları bir veritabanına tablo olarak atabilirler
#daha sonra da power bı benzeri araclarla da erişebilir
#bizim görevimiz segmentlere ayırmak ve dısarı yansıtmak
#rfm olark daha cok tercih edilir.
rfm.to_csv("rfm.csv")

7. Tüm Sürecin Fonksiyonlaştırılması


In [55]:
#aslında biz bir script yazıyoruz ve bunu da bir fonksiyonla yapıyoruz.
def create_rfm(dataframe, csv=False):

    # VERIYI HAZIRLAMA
    dataframe["TotalPrice"] = dataframe["Quantity"] * dataframe["Price"]
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]

    # RFM METRIKLERININ HESAPLANMASI
    today_date = dt.datetime(2011, 12, 11)
    rfm = dataframe.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                                'Invoice': lambda num: num.nunique(),
                                                "TotalPrice": lambda price: price.sum()})
    rfm.columns = ['recency', 'frequency', "monetary"]
    rfm = rfm[(rfm['monetary'] > 0)]

    # RFM SKORLARININ HESAPLANMASI
    rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
    rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
    rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

    # cltv_df skorları kategorik değere dönüştürülüp df'e eklendi
    rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                        rfm['frequency_score'].astype(str))


    # SEGMENTLERIN ISIMLENDIRILMESI
    seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

    rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)
    rfm = rfm[["recency", "frequency", "monetary", "segment"]]
    rfm.index = rfm.index.astype(int)

    if csv:
        rfm.to_csv("rfm.csv")

    return rfm

In [56]:
#veri setini ilk hale getirelim
df=df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [57]:
rfm_new=create_rfm(df)
rfm_new.head()

,recency,frequency,monetary,segment
Customer ID,,,,
12346,530,11,372.860,cant_loose
12347,368,2,1323.320,potential_loyalists
12348,439,1,222.160,hibernating
12349,408,3,2671.140,need_attention
12351,376,1,300.930,new_customers


In [58]:
#fonksyonun ön tanımlı değerinde csv false oldugu için eğer bunu csv istersek belirtelim.
#if csv:  yani varsa
rfm_new=create_rfm(df,csv=True)

In [59]:
rfm_new.head()

,recency,frequency,monetary,segment
Customer ID,,,,
12346,530,11,372.860,cant_loose
12347,368,2,1323.320,potential_loyalists
12348,439,1,222.160,hibernating
12349,408,3,2671.140,need_attention
12351,376,1,300.930,new_customers


In [60]:
#bu bazen değişir her ay çalısması gerekebilir, segmentlerdeki değişiklikleri raporlamak önemlidir.
#alınan aksiyon sonrası takip etme gerekiyor.1 kaç ay sonra aksiyon alınanlara bakılacağı için bitti kenara at diye bi şey yok